### Descarga de librerias y ruta sistema


In [19]:
!sudo apt-get update
!sudo apt-get install tesseract-ocr
!pip install pytesseract

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 3s (148 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [20]:

import os # interactuar con el sistema operativo, como manejar archivos y directorios.
import cv2 # herramientas para procesar y analizar imágenes y videos (OpenCV).
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract' # reconocimiento óptico de caracteres (OCR) en imágenes usando Tesseract.
from PIL import Image # abrir, modificar y guardar imágenes (Pillow)
import csv

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ruta a google drive del csv

In [22]:
IMAGES_FOLDER = "/content/drive/MyDrive/Proyectos/Copalau S-L/4_PROVEEDORES/Programas personales/Fotos programa"
OUTPUT_CSV = "/content/drive/MyDrive/Proyectos/Copalau S-L/4_PROVEEDORES/Programas personales/resultado_OCR.csv"

### Mejorar Calidad Imagen

Para mejorar la imagen debemos:
- Transformarla en escala de grises
- Reducir el ruido con un blur mediano
- Binarizar la imagen (Muy util en imagenes de texto)
- Threshold, separa claro y oscuro.


In [26]:
def mejorar_imagen(ruta_imagen):
    # Cargar en escala de grises
    img = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)

    # Suavizar para reducir ruido
    img = cv2.medianBlur(img, 3)

    # Binarizar con Otsu
    _, img_thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Podrías probar dilatación o erosión si el texto se ve muy fino/grueso.
    return img_thresh

### Procesado de imagenes

Abre la imagen y realiza OCR para extraer su texto, transforma el texto como un string.
    

In [33]:
def procesar_imagen(ruta_imagen):
    # 1. Preprocesar
    img_preprocesada = mejorar_imagen(ruta_imagen)
    # 2. Obtener el texto
    texto = pytesseract.image_to_string(img_preprocesada, lang='eng')  # Usa 'spa' si el texto está mayormente en español
    return texto

In [32]:
import re

def parsear_texto(texto_ocr):
    # Separamos en líneas y quitamos vacías
    lineas = [l.strip() for l in texto_ocr.split('\n') if l.strip()]

    # Variables a extraer
    barcode = ""
    style_no = ""
    eur_price = ""
    talla = ""

    # Buscar código de barras (suele ser solo dígitos largos)
    # Ej: '5714918321434'
    for linea in lineas:
        # Si la línea es sólo dígitos y >= 8-12 chars, puede ser un barcode
        if re.match(r'^\d{8,}$', linea):
            barcode = linea
            break

    # Buscar "STYLE NO." o "STYLE N.O." y quedarnos con el número
    # Ej: 'STYLE NO. 15209614'
    patron_style = re.compile(r'STYLE\s*NO\.?\s*(\d+)', re.IGNORECASE)
    for linea in lineas:
        match = patron_style.search(linea)
        if match:
            style_no = match.group(1)  # Captura el número
            break

    # Buscar precio en EUR, por ejemplo en la línea 'EUR 49.99 DKK 399.95'
    # Podemos buscar algo como: EUR\s+(\d+\.\d+)
    patron_eur = re.compile(r'EUR\s+(\d+\.\d+)', re.IGNORECASE)
    for linea in lineas:
        match = patron_eur.search(linea)
        if match:
            eur_price = match.group(1)
            break

# --- NUEVA SECCIÓN para tallas ---
# Queremos detectar: XS, S, M, L, XL, números 25..44, o bien algo con carácter especial (tipo 27"/32).
    patron_talla = re.compile(
        r'(XS|S|M|L|XL|'       # Tallas de letras
        r'\b2[5-9]\b|'         # 25..29
        r'\b3[0-9]\b|'         # 30..39
        r'\b4[0-4]\b|'         # 40..44
        r'\d+"?/\d+"?)',       # Formato 27/32 o 27"/32
        re.IGNORECASE
        )

    talla = ""  # Iniciamos la variable de talla
    for linea in lineas:
        match = patron_talla.search(linea)
        if match:
            talla = match.group(0)  # por ejemplo "XS", "27/32", "40", etc.
            break #Added break statement to exit loop after finding a match

    return barcode, style_no, eur_price, talla



In [35]:
IMAGES_FOLDER = "/content/drive/MyDrive/Proyectos/Copalau S-L/4_PROVEEDORES/Programas personales/Fotos programa"  # Ajusta a tu carpeta
OUTPUT_CSV = "/content/drive/MyDrive/Proyectos/Copalau S-L/4_PROVEEDORES/Programas personales/resultado_OCR.csv"

def main():
    with open(OUTPUT_CSV, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # Encabezado CSV
        writer.writerow(["Archivo", "Barcode", "StyleNo", "EUR", "Talla"])

        for filename in os.listdir(IMAGES_FOLDER):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                ruta_imagen = os.path.join(IMAGES_FOLDER, filename)

                # OCR
                texto_ocr = procesar_imagen(ruta_imagen)

                # Parseo
                barcode, style_no, eur_price, talla = parsear_texto(texto_ocr)

                # Guardar
                writer.writerow([filename, barcode, style_no, eur_price, talla])

    print("Proceso completado. CSV generado:", OUTPUT_CSV)

if __name__ == "__main__":
    main()

Proceso completado. CSV generado: /content/drive/MyDrive/Proyectos/Copalau S-L/4_PROVEEDORES/Programas personales/resultado_OCR.csv
